# Transformer Model
Training a transformer model to forecast time series sequence of stock closing price  
Using 10 timesteps to forecast 1 forward timestep

Imports

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import sys
import glob
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.signal import savgol_filter
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import KFold
import torch.nn.functional as F
import torch.nn.init as init
import torch.nn.utils as torch_utils
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.multioutput import MultiOutputRegressor
from sklearn.decomposition import KernelPCA
 
import pandas as pd 
import random



C:\Users\benal\AppData\Local\Temp\ipykernel_17608\1405314298.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
#%%
# %%
# INDOORS 1 SUN
file = pd.ExcelFile('Indoor MPP_1.0 sun_Air_ICN2_Feb. 2023_Tripple cat.-Variable Annealing temps.xlsx', engine='openpyxl')# the whole excel file 
sn = file.sheet_names # sheets that are the excel file 
data_frame = pd.read_excel(file,sn[0],  engine='openpyxl') # read the data from the first sheet 

temperature = data_frame.loc[(data_frame["Parameter"]     =="Annealing temperature")].to_numpy() #take the temperature from the row Annealing temperature of column Parameters and convert it to numpy
temperature = temperature[2,2:] # take everything from temperature starting from the third row and the third column
 
devices = data_frame.columns.to_numpy() # take the devices from the data and convert them to numpy
 
devices = np.reshape(devices, (devices.shape[0],1)) 
devices = devices[2:] #excluding the first two elements

time_series_df = pd.read_excel(file,sn[-1],  engine='openpyxl') # read the data from the last sheet

time_series_df = time_series_df.drop(index =[0]) # drop the first row (einheiten)
time = time_series_df.iloc[:,0] # take the time from the first column



# print(time)
time_series_1_sun = []
for i in range (temperature.shape[0]):
    time_series_1_sun.append(time_series_df.iloc[:,2+i*3]) ### for every device take the impp (optimal current output that the solar panel can generate when it is operating at its maximum power point)

time_series_1_sun = np.array(time_series_1_sun) # convert it to array [shape(24, 4001)]


# %% 
# 1.4 SUN




# %%
# %%
# OUTDOOR CONDITIONS
# file = pd.read_csv('outdoor_conditions.txt')
outdoor_conditions = pd.read_csv('outdoor_conditions_unnormalized.txt', sep='\t')


# Save the normalized DataFrame to a new file
# %%
# OUTDOOR PANELS
# file = pd.read_csv('outdoor_conditions.txt')
outdoor_panels = pd.read_csv('outdoor_data_imputed.txt', sep ='\t')
# outdoor_panels.head()
outdoor_panels_power = [] 
for i in range(temperature.shape[0]):
    outdoor_panels_power.append(outdoor_panels[str(i)])
outdoor_panels_power = np.array(outdoor_panels_power) #[shape(24, 4794)]

#%%
# Padding


time_series_1_sun_shape = time_series_1_sun.shape
 
time_series_1_sun = np.pad(time_series_1_sun, ((0,0),(0,outdoor_panels_power.shape[1] - time_series_1_sun.shape[1])),mode='edge')    
    
time_series_1_sun = np.reshape(time_series_1_sun, ((time_series_1_sun.shape[0],time_series_1_sun.shape[1],1)))
 
outdoor_panels_power = np.reshape(outdoor_panels_power, ((outdoor_panels_power.shape[0],outdoor_panels_power.shape[1],1)))


In [3]:
# Calculate average values for time_series_1_sun
time_series_1_sun_average = np.zeros((12, time_series_1_sun.shape[1], 1))
j = 0
for i in range(time_series_1_sun.shape[0]):
    if (i+1) % 2 == 0:
        time_series_1_sun_average[j] = np.mean(time_series_1_sun[i-1:i+1], axis=0)
        j += 1
 
     

# Calculate average values for time_series_1_4_sun


# Calculate average values for outdoor_panels_power
outdoor_panels_power_average = np.zeros((12, 4794, 1))
b = 0
for i in range(outdoor_panels_power.shape[0]):
    if (i+1) % 2 == 0:
        outdoor_panels_power_average[b] = np.mean(outdoor_panels_power[i-1:i+1], axis=0)
        outdoor_panels_power_average[b][outdoor_panels_power_average[b] < 0] = 0 # Set negative values to 0
        b += 1

# Stack the outdoor conditions
outdoor_conditions_stacked = np.stack([outdoor_conditions ] * 12, axis=0)
outdoor_conditions_stacked = outdoor_conditions_stacked[:,:,2:]

print(outdoor_conditions_stacked[0,0,:])
# outdoor_conditions_stacked_2= np.stack([outdoor_conditions] * 20, axis=0)

[51.26533 56.629   18.8651 ]


In [4]:
def exponential_smoothing(series, alpha):
    smoothed_series = [series[0]]  # Initialize the smoothed series with the first value of the original series
    for i in range(1, len(series)):
        smoothed_value = alpha * series[i] + (1 - alpha) * smoothed_series[-1]
        smoothed_series.append(smoothed_value)
    return np.array(smoothed_series)
 
alpha = 0.1   
for i in range (time_series_1_sun_average.shape[0]):
 
  time_series_1_sun_average[i] = exponential_smoothing(time_series_1_sun_average[i], 0.1)
  outdoor_panels_power_average[i] = exponential_smoothing(outdoor_panels_power_average[i], 0.7)
all_new_features = np.concatenate((time_series_1_sun_average , outdoor_conditions_stacked), axis=2)
print(all_new_features.shape)
print(outdoor_panels_power_average.shape)


#TODO REMOVE PADDING SEE IF IT BREAKS

(12, 4794, 4)
(12, 4794, 1)


In [5]:
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('CUDA is available. Using GPU for computations.')
else:
    device = torch.device('cpu')
    print('CUDA is not available. Using CPU for computations.')
device = 'cpu'

CUDA is not available. Using CPU for computations.


Load data and set global variables

Plot shows the reduced scale of the closing prices

Positional Encoder

In [6]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]        

Transformer Model

In [7]:
class TransAm(nn.Module):
    def __init__(self, feature_size=4, num_layers=1, dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=2, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)  # TODO ADD A BETTER NN WITH ACTIVATIONS AND STUFF
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0) != len(src) or not torch.is_tensor(self.src_mask):
#             device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
#             mask= torch.from_numpy(mask)
            self.src_mask = mask

        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [13]:
input_dim = 6  # number of features in the outdoor conditions and the indoor measurements
hidden_dim = 128
output_dim = 1  # the number of features in the output
num_epochs = 2
num_layers = 1
num_mixtures = 5
learning_rate = 0.001
weight_decay = 0.001
Fold = 0
Predictions = np.zeros((6, 4794, 2))
Test = np.zeros((6, 4794, 2))
kf = KFold(n_splits=12)


num_samples = all_new_features.shape[0]


 
all_new_features=all_new_features[0:12, :, :].astype(np.float32)
all_new_featurest= torch.from_numpy(all_new_features).float().to(device)
outdoor_panels_power_averaget= torch.from_numpy(outdoor_panels_power_average[0:12, :, :]).float().to(device)

# Define the number of folds for cross-validation
num_folds = 12

# Initialize lists to store the average MSE for each fold
average_mse_scores1 = []
average_mse_scores2 = []
fold = 0
y_pred1=np.zeros(( 1, 4794))
y_pred2=np.zeros(( 1, 4794))
# Initialize arrays to store the predictions and test data for each fold
predictions1 = np.zeros((12, 1, 4794))
test1= np.zeros((12, 1, 4794))
predictions2 = np.zeros((12, 1, 4794))
test2 = np.zeros((12, 1, 4794))
# Perform k-fold cross-validation
 
for temp in range(6):
    test_index=[2*temp,2*temp+1]
    train_index = [i for i in range(num_samples) if i not in test_index]
    # Split the data into training and test sets for the current fold
    X_train, X_test = all_new_featurest[train_index], all_new_featurest[test_index]
    y_train, y_test = outdoor_panels_power_averaget[train_index], outdoor_panels_power_averaget[test_index]
    model = TransAm(feature_size=4, num_layers=15, dropout=0.5).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    # Training loop
    for epoch in range(1):
        model.train()  # Set the model in training mode
        outputs = model(X_train)
        outputs = outputs.view(X_train.shape[0], 4794, 1)
        loss = criterion(outputs, y_train)
        optimizer.zero_grad()
        loss.backward()
        torch_utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()

    model.eval()  # Set the model in evaluation mode for testing
    with torch.no_grad():
        sample_input = X_test
        predictions = model(sample_input)
        predictions_temp = predictions
        predictions_temp = predictions_temp[:,:,0].T
        Predictions[Fold] = predictions_temp.cpu().numpy()
        y_test_temp = y_test

        y_test_temp = y_test_temp[:,:,0].T
        Test[Fold] = y_test_temp.cpu().numpy()

        mse = F.mse_loss(predictions, y_test)
        print("Mean Squared Error (MSE):", mse.item())
        y_test_temp = y_test_temp.cpu()
        predictions_temp = predictions_temp.cpu()
#         Plotting the predictions against the actual values
        x = range(y_test_temp.T.shape[1])
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        for i, axs in enumerate(axs.flat):
            print(i)
            axs.plot(x, y_test_temp.T[i].flatten(), label='Actual')
            axs.plot(x, predictions_temp.T[i].flatten(), label='Prediction', color='red')
            axs.set_xlabel('Time')
            axs.set_ylabel('Value')
            axs.legend()
        fig.suptitle('Predictions vs Actual Values')
        plt.tight_layout()
        plt.show()

        Fold += 1

        # If Fold reaches 2, stop the training loop (for demonstration purposes)
        if Fold == 6:
            break

KeyboardInterrupt: 

Window function, split data into sequence window

Split data in training and testing, prepared in windowed sequences and pass through GPU

Split into training batches

Training function

Evaluation function for model after training

In [9]:
def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i, eval_batch_size)
            output = eval_model(data)            
            total_loss += len(data[0])* criterion(output, targets).cpu().item()
    return total_loss / len(data_source)

Function to forecast 1 time step from window sequence

In [10]:
def model_forecast(model, seqence):
    model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)

    seq = np.pad(seqence, (0, 3), mode='constant', constant_values=(0, 0))
    seq = create_inout_sequences(seq, input_window)
    seq = seq[:-output_window].to(device)

    seq, _ = get_batch(seq, 0, 1)
    with torch.no_grad():
        for i in range(0, output_window):            
            output = model(seq[-output_window:])                        
            seq = torch.cat((seq, output[-1:]))

    seq = seq.cpu().view(-1).numpy()

    return seq

Function to forecast entire sequence

In [11]:
def forecast_seq(model, sequences):
    """Sequences data has to been windowed and passed through device"""
    start_timer = time.time()
    model.eval() 
    forecast_seq = torch.Tensor(0)    
    actual = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(sequences) - 1):
            data, target = get_batch(sequences, i, 1)
            output = model(data)            
            forecast_seq = torch.cat((forecast_seq, output[-1].view(-1).cpu()), 0)
            actual = torch.cat((actual, target[-1].view(-1).cpu()), 0)
    timed = time.time()-start_timer
    print(f"{timed} sec")

    return forecast_seq, actual

Prepare data for training model

In [12]:
train_data, val_data = get_data(logreturn, 0.6) # 60% train, 40% test split
model = TransAm().to(device)

NameError: name 'get_data' is not defined

Model parameters

In [ ]:
criterion = nn.MSELoss() # Loss function
lr = 0.00005 # learning rate

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

epochs =  5 # Number of epochs

Training loop

In [ ]:
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data)
    
    if(epoch % epochs == 0): # Valid model after last training epoch
        val_loss = evaluate(model, val_data)
        print('-' * 80)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss: {:5.7f}'.format(epoch, (time.time() - epoch_start_time), val_loss))
        print('-' * 80)

    else:   
        print('-' * 80)
        print('| end of epoch {:3d} | time: {:5.2f}s'.format(epoch, (time.time() - epoch_start_time)))
        print('-' * 80)

    scheduler.step() 


In [ ]:
test_result, truth = forecast_seq(model, val_data)

Plot forecasted sequence vs actual

Test random sequence

Save model

Load model

Testing model on Boeing stock from the same time period

Testing model on JPMorgan stock from the same time period

## Conclusion
Transformer model is able to train on a longer sequence compared to LSTM models (5 vs 10)  
Transformer model also trained faster, given the longer sequence length, and took lesser epochs to train the model

### Data Transforming and Normalising
Advantages of using cumulative sum of log returns compared to min-max scaler in normalising stocks prices,  
create better standardisation across stocks  
Training data augmentation allowed model to be trained on wider data points, resulting in the model generalising well across unseen data from test 2 and test 3  
Previous testing without data augmentation had models underperforming with higher loss